# Businesses - Brand and Competition Locations

The geography level methods and enrichment methods detailed in the previous notebooks enable retrieving location and demographic information describing where and who customers are. Next though, we need to retrieve information about the retail landscape surrounding these potential customers. Knowing the locations of brand and competitor locations is essential to modeling the effects of competition and cannibalization.

In [1]:
from pathlib import Path
import sys

project_parent = Path('./').absolute().parent
dir_data = project_parent/'data'
dir_int = dir_data/'interim'
gdb_int = dir_int/'interim.gdb'

# import the project package from the project package path
# ideally will be imported using 'from arcgis import dm'
sys.path.append(str(project_parent/'src'))
import dm

# load the "autoreload" extension so as src code is changed, the changes are picked up in the dataframe
%load_ext autoreload
%autoreload 2

Getting started is not dramactically different than before - instantiate a country object intsance and delineate an area of interest.

In [2]:
usa = dm.Country('USA')
aoi_df = usa.cbsas.get('seattle')

aoi_mp = aoi_df.spatial.plot()
aoi_mp.basemap = 'gray-vector'
aoi_mp

MapView(layout=Layout(height='400px', width='100%'))

## Brand Locations

Then, business locations can be retrieved using the business name.

In [3]:
biz_df = usa.business.get_by_name('Ace Hardware', aoi_df)
biz_df.head()

,OBJECTID,LOCNUM,CONAME,STREET,CITY,STATE,STATE_NAME,ZIP,ZIP4,NAICS,...,SQFTCODE,LOC_NAME,STATUS,SCORE,SOURCE,REC_TYPE,SHAPE,id,brand_name,brand_name_category
0,1742102,371889957,GRAHAM ACE HARDWARE,224TH ST E,GRAHAM,WA,Washington,98338,5704,44413005,...,5,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -122.29909999999992, ""y"": 47.05410000000...",371889957,GRAHAM ACE HARDWARE,GRAHAM ACE HARDWARE
1,5853272,460556608,OAKBROOK ACE HARDWARE,STEILACOOM BLVD SW,LAKEWOOD,WA,Washington,98498,6154,44413005,...,3,StreetAddress,M,100.0,INFOGROUP,0,"{""x"": -122.5456, ""y"": 47.18000000000013, ""spat...",460556608,OAKBROOK ACE HARDWARE,OAKBROOK ACE HARDWARE
2,2830418,405129289,GIG HARBOR ACE HARDWARE,POINT FOSDICK DR NW,GIG HARBOR,WA,Washington,98335,1711,44413005,...,7,StreetAddress,M,100.0,INFOGROUP,0,"{""x"": -122.57929999999999, ""y"": 47.30140000000...",405129289,GIG HARBOR ACE HARDWARE,GIG HARBOR ACE HARDWARE
3,2690144,404324160,AGRISHOP ACE HARDWARE,S 12TH ST,TACOMA,WA,Washington,98405,3004,44413005,...,5,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -122.46379999999998, ""y"": 47.24990000000...",404324160,AGRISHOP ACE HARDWARE,AGRISHOP ACE HARDWARE
4,2215877,401684669,ACE HARDWARE & PAINT CTR,N PEARL ST,TACOMA,WA,Washington,98407,2416,44413005,...,8,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -122.51369999999999, ""y"": 47.27360000000...",401684669,ACE HARDWARE & PAINT CTR,ACE HARDWARE & PAINT CTR


In [4]:
biz_mp = biz_df.spatial.plot()
biz_mp.basemap = 'gray-vector'
biz_mp

MapView(layout=Layout(height='400px', width='100%'))

Businesses can also be looked up by their category code. By default, this uses NAICS, but SIC can also be used. Also, something very useful for chains, especially QSR's, is to identify local brands collectively. This is calculated based on the company name, how many times the company name appears. In this case, the method does not work as well since every Ace Hardware has a unique name based on the location.

In [5]:
naics_df = usa.business.get_by_code(44413005, aoi_df, local_threshold=2)

print(f'rows retrieved: {len(naics_df.index)}\n')

naics_df.head()

rows retrieved: 128



,OBJECTID,LOCNUM,CONAME,STREET,CITY,STATE,STATE_NAME,ZIP,ZIP4,NAICS,...,SQFTCODE,LOC_NAME,STATUS,SCORE,SOURCE,REC_TYPE,SHAPE,id,brand_name,brand_name_category
0,1742102,371889957,GRAHAM ACE HARDWARE,224TH ST E,GRAHAM,WA,Washington,98338,5704,44413005,...,5,PointAddress,M,100.0000,INFOGROUP,0,"{""x"": -122.29909999999992, ""y"": 47.05410000000...",371889957,GRAHAM ACE HARDWARE,local_brand
1,11948738,740405810,CASCADE WHOLESALE HARDWARE,PACIFIC HWY SW,LAKEWOOD,WA,Washington,98499,1027,44413005,...,3,PointAddress,M,100.0000,INFOGROUP,0,"{""x"": -122.51639999999995, ""y"": 47.14270000000...",740405810,CASCADE WHOLESALE HARDWARE,local_brand
2,791119,175096239,LAKEWOOD HARDWARE & PAINT,LAKEWOOD DR SW,LAKEWOOD,WA,Washington,98499,3966,44413005,...,5,PointAddress,M,100.0000,INFOGROUP,0,"{""x"": -122.50849999999993, ""y"": 47.17530000000...",175096239,LAKEWOOD HARDWARE & PAINT,local_brand
3,5853272,460556608,OAKBROOK ACE HARDWARE,STEILACOOM BLVD SW,LAKEWOOD,WA,Washington,98498,6154,44413005,...,3,StreetAddress,M,100.0000,INFOGROUP,0,"{""x"": -122.5456, ""y"": 47.18000000000013, ""spat...",460556608,OAKBROOK ACE HARDWARE,local_brand
4,9170068,714960080,OAKBROOK ACE,ISTEILACOOM BLVD SW,LAKEWOOD,WA,Washington,98498,,44413005,...,3,StreetAddress,M,90.9063,INFOGROUP,0,"{""x"": -122.5456, ""y"": 47.18000000000013, ""spat...",714960080,OAKBROOK ACE,local_brand


Further, it frequently is also useful to retrieve broader categories by simply truncating two digits at a time to be less specific.

In [6]:
naics_big_df = usa.business.get_by_code(4441, aoi_df, local_threshold=2)

print(f'rows retrieved: {len(naics_big_df.index):,}\n')

naics_big_df.head()

rows retrieved: 1,664



,OBJECTID,LOCNUM,CONAME,STREET,CITY,STATE,STATE_NAME,ZIP,ZIP4,NAICS,...,SQFTCODE,LOC_NAME,STATUS,SCORE,SOURCE,REC_TYPE,SHAPE,id,brand_name,brand_name_category
0,964999,218001303,TIN MEN SUPPLY INC,STATE ROUTE 507 S,ROY,WA,Washington,98580,,44419055,...,3,StreetAddress,T,100.0,INFOGROUP,0,"{""x"": -122.55289999999998, ""y"": 46.94250000000...",218001303,TIN MEN SUPPLY INC,local_brand
1,6036182,481554129,MILES SAND & GRAVEL CO,STATE ROUTE 507 S,ROY,WA,Washington,98580,8745,44419012,...,5,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -122.5515, ""y"": 46.97810000000003, ""spat...",481554129,MILES SAND & GRAVEL CO,local_brand
2,12885902,961135167,MC KENNA DOOR,46TH AVE S,ROY,WA,Washington,98580,7623,44419017,...,2,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -122.49179999999998, ""y"": 46.93930000000...",961135167,MC KENNA DOOR,local_brand
3,12707346,904945607,GENERATION GLASS CO INC,8TH AVE S,ROY,WA,Washington,98580,0018,44419031,...,2,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -122.44390000000001, ""y"": 46.89830000000...",904945607,GENERATION GLASS CO INC,local_brand
4,12978535,973622731,ARROW LUMBER & HARDWARE,CENTER ST E,EATONVILLE,WA,Washington,98328,7449,44419044,...,5,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -122.26169999999995, ""y"": 46.86610000000...",973622731,ARROW LUMBER & HARDWARE,ARROW LUMBER & HARDWARE


## Competitor Locations

Also, competitors can easily be located using the previously retrieved data, or using a brand name to exclude from retrieved results. The latter is extremely useful when using locations the organization already has for the brand locations, but needing competitor locations for modeling.

In [7]:
# comp_df = usa.business.get_competition(biz_df, aoi_df)
comp_df = usa.business.get_competition('Ace Hardware', aoi_df, local_threshold=2)

comp_df.head()

,OBJECTID,LOCNUM,CONAME,STREET,CITY,STATE,STATE_NAME,ZIP,ZIP4,NAICS,...,SQFTCODE,LOC_NAME,STATUS,SCORE,SOURCE,REC_TYPE,SHAPE,id,brand_name,brand_name_category
0,11948738,740405810,CASCADE WHOLESALE HARDWARE,PACIFIC HWY SW,LAKEWOOD,WA,Washington,98499,1027,44413005,...,3,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -122.51639999999995, ""y"": 47.14270000000...",740405810,CASCADE WHOLESALE HARDWARE,local_brand
1,791119,175096239,LAKEWOOD HARDWARE & PAINT,LAKEWOOD DR SW,LAKEWOOD,WA,Washington,98499,3966,44413005,...,5,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -122.50849999999993, ""y"": 47.17530000000...",175096239,LAKEWOOD HARDWARE & PAINT,local_brand
2,9459298,717157403,SWINGS & THINGS,BRIDGEPORT WAY SW,LAKEWOOD,WA,Washington,98499,2645,44413005,...,1,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -122.5175, ""y"": 47.17680000000013, ""spat...",717157403,SWINGS & THINGS,local_brand
3,9417066,716987694,WILCO FARM STORES-GIG HARBOR,E HIGHWAY 20,GIG HARBOR,WA,Washington,98335,,44413005,...,3,Postal,M,100.0,INFOGROUP,0,"{""x"": -122.57939999999995, ""y"": 47.30740000000...",716987694,WILCO FARM STORES-GIG HARBOR,local_brand
4,4190467,425066759,MCLENDON HARDWARE INC,N PEARL ST,TACOMA,WA,Washington,98406,7921,44413005,...,7,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -122.51449999999988, ""y"": 47.25920000000...",425066759,MCLENDON HARDWARE INC,MCLENDON HARDWARE INC


Also, just as before, the returned results are Spatially Enabled DataFrames, so they can be easily saved for follow on analysis or visualization in ArcGIS Pro.

In [8]:
biz_df.spatial.to_featureclass(gdb_int/'loc_brand')

'D:\\projects\\demographic-modeling-module\\data\\interim\\interim.gdb\\loc_brand'

In [9]:
comp_df.spatial.to_featureclass(gdb_int/'loc_comp')

'D:\\projects\\demographic-modeling-module\\data\\interim\\interim.gdb\\loc_comp'